In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self):
        super(EncoderBlock, self).__init__()
        nn.Sequential(
                ('EncoderBlock', 
                 nn.Sequential(
                    ('ln_1', nn.LayerNorm(normalized_shape=(768,), eps=1e-6, elementwise_affine=True)),
                    ('self_attention', nn.MultiheadAttention(embed_dim=768, num_heads=12, dropout=0.0)),),
                    ('dropout', nn.Dropout()),
                    ('ln_2', nn.LayerNorm(normalized_shape=(768,), eps=1e-6, elementwise_affine=True)),
                    ('mlp', 
                     nn.Sequential(
                        nn.Linear(768, 3072, bias=True),
                        nn.GELU(approximate='none'),
                        nn.Dropout(p=0.0, inplace=False),
                        nn.Linear(3072, 768, bias=True),
                        nn.Dropout(p=0.0, inplace=False),
                        )
                    )
                )
            )
        )

class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(Encoder, self).__init__()
        self.layers = nn.Sequential()
        
        self.MultiheadAttention = None
        self.MLPBlock = None

class ViT(nn.Module):
    def __init__(self):
        super(ViT, self).__init__()
        # self.vit = torchvision.models.vit.vit_base_patch16_224_in21k(pretrained=True)
        self.conv = nn.Conv2d(in_channels=3, out_channels=768, kernel_size=16, stride=16)

##### ViT -- PyTorch Version

In [4]:
vit_pt = torchvision.models.vit_b_16(pretrained=True)
vit_pt

/Users/jcus/Documents/vmv/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jcus/Documents/vmv/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /Users/jcus/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:08<00:00, 38.5MB/s] 


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [11]:
vit_pt.__dict__.keys()

dict_keys(['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_pre_hooks', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_hooks_always_called', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_state_dict_hooks', '_state_dict_pre_hooks', '_load_state_dict_pre_hooks', '_load_state_dict_post_hooks', '_modules', 'image_size', 'patch_size', 'hidden_dim', 'mlp_dim', 'attention_dropout', 'dropout', 'num_classes', 'representation_size', 'norm_layer', 'seq_length'])

In [49]:
from PIL import Image
import numpy as np
import torch

img = Image.open('../../ICBC-CV/meeting.jpeg')
img = img.resize((224, 224))
width, height = img.size
img = np.array(img)

img = np.transpose(img, (2, 0, 1))
img_arr = torch.tensor(img, dtype=torch.float32)
img_arr = img_arr.unsqueeze(0)
# img_arr
# img_arr.shape
pred = vit_pt(img_arr)

tensor(431)